<a href="https://colab.research.google.com/github/vdovych/nlp_js_project/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

__need__:

accuracy precision recall

звяти верхню нижню квантиль попробувати

__would be good__

grid search

__experiment__

optimizer=ks.optimizers.SGD(lr=0.01)

<!-- try l2 regularizattion
 -->
learning rate

more layers?

### data mixing & preprocessing

In [0]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


In [0]:
gen_target = pd.read_csv('bbc_target.csv', header=None)[0].append(pd.Series(data=fetch_20newsgroups(subset='all').target), ignore_index=True)
gen_text = pd.read_csv('bbc_text.csv', header=None)[0].append(pd.Series(data=fetch_20newsgroups(subset='all', remove=('headers', 'footers')).data), ignore_index=True)

In [0]:
gen_target.replace({'business':20, 'entertainment':21, 'politics':22, 'sport':23, 'tech':24 }, inplace=True)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(gen_text, gen_target)

In [0]:
features.shape

(15802, 19639)

In [0]:
# ['alt.atheism',0,0
#  'comp.graphics',1,1
#  'comp.os.ms-windows.misc',2,1
#  'comp.sys.ibm.pc.hardware',3,1
#  'comp.sys.mac.hardware',4,1
#  'comp.windows.x',5,1
#  'misc.forsale',6,1
#  'rec.autos',7,0
#  'rec.motorcycles',8,0
#  'rec.sport.baseball',9.0
#  'rec.sport.hockey',10,0
#  'sci.crypt',11,1
#  'sci.electronics',12,1
#  'sci.med',13,1
#  'sci.space',14,1
#  'soc.religion.christian',15,0
#  'talk.politics.guns',16,0
#  'talk.politics.mideast',17,0
#  'talk.politics.misc',18,0
#  'talk.religion.misc',19,0]
#  'business',20, 1
#  'entertainment',21, 0
#  'politics',22, 0
#  'sport',23, 0
#  'tech',24, 1

tech = [1,2,3,4,5,6,11,12,13,14,20,24]


In [0]:
bin_tr_y = np.isin(y_train.values, tech).astype(np.uint8)
bin_tes_y = np.isin(y_test.values, tech).astype(np.uint8)

In [0]:
vectors_test = tfidf.transform(x_test)
# np.savetxt('test_x', vectors_test.toarray()[:2])
clf = MultinomialNB(alpha=.01)
clf.fit(features, bin_tr_y)
pred = clf.predict(vectors_test)
accuracy_score(bin_tes_y, pred)

0.9502657555049354

In [0]:
unique, counts = np.unique(bin_tr_y, return_counts=True)
dict(zip(unique, counts))

{0: 8459, 1: 7343}

### Model & train

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
import keras

In [0]:
simple_model = Sequential([Dense(100,activation='relu', input_dim=features.shape[1], kernel_regularizer=l2(0.01)),
                           Dropout(0.5, noise_shape=None, seed=None),
                           Dense(100,activation='relu', kernel_regularizer=l2(0.01)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(1,activation='sigmoid')])

ES_callback = EarlyStopping(monitor='val_loss', min_delta=0.00005, patience=4)

# define the checkpoint
filepath="aaaaaaaaaaaaaaaaaaaaaaweights-improvement-{epoch:02d}-{loss:.4f}-{val_acc:.4f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

In [0]:
simple_model.compile(loss='binary_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])

In [0]:
simple_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               1964000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 1,974,201
Trainable params: 1,974,201
Non-trainable params: 0
_________________________________________________________________


In [0]:
model_output= simple_model.fit(features,
                               bin_tr_y,
                               batch_size=256,
                               epochs=40,
                               verbose=1,
                               validation_data=(vectors_test, bin_tes_y),
                               callbacks=[ES_callback, checkpoint])

Train on 15802 samples, validate on 5268 samples
Epoch 1/40
15802/15802 [==============================] - 3s 210us/step - loss: 1.6284 - acc: 0.8908 - val_loss: 0.9322 - val_acc: 0.9645

Epoch 00001: loss improved from inf to 1.62845, saving model to aaaaaaaaaaaaaaaaaaaaaaweights-improvement-01-1.6284-0.9645.hdf5
Epoch 2/40
15802/15802 [==============================] - 3s 170us/step - loss: 0.6973 - acc: 0.9705 - val_loss: 0.5729 - val_acc: 0.9630

Epoch 00002: loss improved from 1.62845 to 0.69730, saving model to aaaaaaaaaaaaaaaaaaaaaaweights-improvement-02-0.6973-0.9630.hdf5
Epoch 3/40
15802/15802 [==============================] - 3s 170us/step - loss: 0.4573 - acc: 0.9743 - val_loss: 0.4533 - val_acc: 0.9541

Epoch 00003: loss improved from 0.69730 to 0.45731, saving model to aaaaaaaaaaaaaaaaaaaaaaweights-improvement-03-0.4573-0.9541.hdf5
Epoch 4/40
15802/15802 [==============================] - 3s 171us/step - loss: 0.4043 - acc: 0.9715 - val_loss: 0.4020 - val_acc: 0.9599

Epo

#### Saving the most successful weighted model

In [0]:
simple_model = Sequential([Dense(100,activation='relu', input_dim=features.shape[1], kernel_regularizer=l2(0.01)),
                           Dropout(0.5, noise_shape=None, seed=None),
                           Dense(100,activation='relu', kernel_regularizer=l2(0.01)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(1,activation='sigmoid')])

simple_model.load_weights('aaaaaaaaaaaaaaaaaaaaaaweights-improvement-01-1.5846-0.9592.hdf5')

simple_model.compile(loss='binary_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])

In [0]:
!pip install tensorflowjs

    100% |████████████████████████████████| 61kB 4.8MB/s 
    100% |████████████████████████████████| 307kB 8.0MB/s 
    100% |████████████████████████████████| 13.9MB 3.4MB/s 
    100% |████████████████████████████████| 51kB 25.9MB/s 
yellowbrick 0.9 has requirement matplotlib<3.0,>=1.5.1, but you'll have matplotlib 3.0.2 which is incompatible.
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.
tensorflow 1.12.0 has requirement keras-applications>=1.0.6, but you'll have keras-applications 1.0.4 which is incompatible.
tensorflow 1.12.0 has requirement keras-preprocessing>=1.0.5, but you'll have keras-preprocessing 1.0.2 which is incompatible.
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.1 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatibl

In [0]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(simple_model, 'model_stillnotfat')

#### for js code testing

In [0]:
text = ["Redux DevTools for debugging application's state changes.", "I want to use this classifier for a commercial project.", "for debugging application's I want to use this classifier"]

In [0]:
tfidf = TfidfVectorizer(stop_words='english', norm=None, use_idf=True, smooth_idf=True, sublinear_tf=True)
feat = tfidf.fit_transform(text)

In [0]:
simple_model = Sequential([Dense(100,activation='relu', input_dim=11, kernel_regularizer=l2(0.01)),
                           Dropout(0.5, noise_shape=None, seed=None),
                           Dense(100,activation='relu', kernel_regularizer=l2(0.01)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(1,activation='sigmoid')])

ES_callback = EarlyStopping(monitor='val_loss', min_delta=0.00005, patience=4)

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-{acc:.4f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

In [0]:
simple_model.compile(loss='binary_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])

In [0]:
model_output= simple_model.fit(feat,
                               [1,0,0],
                               verbose=1,
                               callbacks=[ES_callback, checkpoint])

Epoch 1/1
3/3 [==============================] - 2s 778ms/step - loss: 1.8090 - acc: 0.6667

Epoch 00001: loss improved from inf to 1.80903, saving model to weights-improvement-01-1.8090-0.6667.hdf5


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


#### Manual trial

In [0]:
# text = 'i want to use this classifier for a commercial project. It is light-weighted and can work in a chrome plugin'
# text = 'i care for my country very much, vote for me. believeme, i will be the best president you have ever known'
# text = 'This ballet style is often performed barefoot. Contemporary ballets may include mime and acting, and are usually set to music (typically orchestral but occasionally vocal). It can be difficult to differentiate this form from neoclassical or modern ballet. Contemporary ballet is also close to contemporary dance, because many contemporary ballet concepts come from the ideas and innovations of twentieth-century modern dance, including floor work and turn-in of the legs. The main distinction is that ballet technique is essential to perform a contemporary ballet.George Balanchine is considered to have been a pioneer of contemporary ballet. Another early contemporary ballet choreographer, Twyla Tharp, choreographed Push Comes To Shove for the American Ballet Theatre in 1976, and in 1986 created In The Upper Room for her own company. Both of these pieces were considered innovative for their melding of distinctly modern movements with the use of pointe shoes and classically trained dancers.Today there are many contemporary ballet companies and choreographers. These include Alonzo King and his company LINES Ballet; Matthew Bourne and his company New Adventures; Complexions Contemporary Ballet; Nacho Duato and his Compañia Nacional de Danza; William Forsythe and The Forsythe Company; and Jiří Kylián of the Nederlands Dans Theater. Traditionally "classical" companies, such as the Mariinsky (Kirov) Ballet and the Paris Opera Ballet, also regularly perform contemporary works.The term ballet has evolved to include all forms associated with it. Someone training as a ballet dancer will now be expected to perform neoclassical, modern and contemporary work. A ballet dancer is expected to be able to be stately and regal for classical work, free and lyrical in neoclassical work, and unassuming, harsh or pedestrian for modern and contemporary work. In addition, there are several modern varieties of dance that fuse classical ballet technique with contemporary dance, such as Hiplet, that require dancers to be practised in non-Western dance styles.'
# text='The first thing that comes into our minds while thinking of employees at tech companies is probably Software Engineers and Data Scientists. But a technology company requires employees beyond those in technical roles, so they hire for non-tech roles too. The current scenario at the job market doesn’t necessarily require you to be a tech pro to get hired by technology companies. As the growth tech companies increases, they need more non-technical workforce to help transform their technology into revenue. Roles such as sales, marketing, business operation, etc. are crucial to their expansion. These highly coveted positions are also well compensated by tech employers.'
# # text = '''As part of an effort to reduce noise in Ghana’s capital, Accra, the environment minister has suggested that the Muslim call to prayer, normally broadcast over loudspeakers across the city, should instead be sent out on WhatsApp. The notion has proved immensely unpopular – not least because it equates the call to prayer with noise pollution. But it also highlights religion’s growing, if sometimes uneasy, reliance on tech.

# Contactless collection Catholic and Protestant churches in the UK have begun using contactless card readers for donations and other payments, hoping to make life easier for parishioners who may not be carrying cash. A contactless collection plate is being trialled by the Church of England, but it is being held up because it is feared the technology might slow things down.

# The confession app Confession (version 2.1) walks sinners through the business of confession, pings you push notifications when it’s time for your next shriving and includes a handy sin checklist in case you have forgotten what you’ve done wrong. What it doesn’t do is offer absolution. You still need a priest for that.

# Televangelist tech American TV preachers tend to be at home in the digital realm, particularly those espousing “prosperity theology”, which holds that God rewards the pious with material wealth. It works for them, anyway. Controversial televangelist Joel Osteen’s free app offers daily devotional messages and sermons. And Creflo Dollar’s latest app boasts a “give” feature that makes handing your money over to Creflo A Dollar Ministries as easy as gambling. Webcast rites Funerals broadcast online are at least a decade old. Many UK crematoria now offer it as part of the package, and rabbis and vicars routinely officiate at live-streamed services. It is not quite the same as being there, but in many cases it’s probably preferable.Halal apps There are several apps that help Muslims find halal restaurants and stores in the immediate area . There is even one – Scan Halal – that allows you to scan the barcode of any food in a supermarket to instantly determine whether it contains non-halal ingredients. Online reviews for this idea are decidedly mixed.'
# '''
# text = "Redux Redux DevTools debugging"
text = "Redux DevTools for debugging application's state changes."


In [0]:
transformed = tfidf.transform([text])

In [0]:
tfidf._tfidf._idf_diag.toarray()

array([[1.28768207, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 1.69314718, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.28768207, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 1.69314718, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.28768207,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.69314718, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.

In [0]:
transformed.toarray().tolist()

[[1.2876820724517808,
  1.6931471805599454,
  0.0,
  0.0,
  1.2876820724517808,
  1.6931471805599454,
  0.0,
  1.6931471805599454,
  1.6931471805599454,
  0.0,
  0.0]]

In [0]:
tfidf.idf_.tolist()

[1.2876820724517808,
 1.6931471805599454,
 1.2876820724517808,
 1.6931471805599454,
 1.2876820724517808,
 1.6931471805599454,
 1.6931471805599454,
 1.6931471805599454,
 1.6931471805599454,
 1.2876820724517808,
 1.2876820724517808]

In [0]:
tfidf.vocabulary_

{'application': 0,
 'changes': 1,
 'classifier': 2,
 'commercial': 3,
 'debugging': 4,
 'devtools': 5,
 'project': 6,
 'redux': 7,
 'state': 8,
 'use': 9,
 'want': 10}

In [0]:
simple_model.predict(transformed)


array([[0.49849725]], dtype=float32)

#### Save data for further tf-idf vectorization

In [0]:
import json

In [0]:
def default(o):
    if isinstance(o, np.int64): return int(o)  
    if isinstance(o, np.float64): return float(o)
    raise TypeError

In [0]:
with open('tfidf_vocab.json', 'w') as f:
    json.dump(tfidf.vocabulary_, f, default=default)

In [0]:
with open('idf_.json', 'w') as f:
    json.dump(tfidf.idf_.tolist(), f)

In [0]:
with open('transformed.json', 'w') as f:
    json.dump(transformed.toarray().tolist(), f)

In [0]:
with open('idf_.json') as f:
    data=json.load(f)data

[4.6324736418843475, 3.958650734150805, 8.270059801610733, 7.6722228008551125, 8.470730497072886, 7.895366352169322, 8.183048424621104, 8.470730497072886, 5.442208400695902, 5.316096940933037, 8.270059801610733, 8.183048424621104, 5.726312651799799, 7.777583316512939, 8.470730497072886, 5.592781259175277, 7.11260701291969, 8.103005716947568, 5.624529957489857, 8.270059801610733, 6.225303817918787, 8.270059801610733, 6.114078182808563, 8.470730497072886, 6.093244095905721, 8.470730497072886, 8.470730497072886, 8.470730497072886, 5.995126239947197, 8.365369981415057, 8.365369981415057, 8.028897744793845, 8.183048424621104, 8.365369981415057, 8.028897744793845, 8.183048424621104, 8.103005716947568, 8.470730497072886, 8.103005716947568, 7.4899012440611585, 8.183048424621104, 8.365369981415057, 7.959904873306893, 8.365369981415057, 8.365369981415057, 7.895366352169322, 8.470730497072886, 8.365369981415057, 8.183048424621104, 8.470730497072886, 8.103005716947568, 3.386569410838821, 4.1254831